In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import os
import math
import matplotlib.pyplot as plt
import numpy as np

from Load_Data import load_data

device = (
    "mps"
    if torch.backends.mps.is_available()
    else "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
device = torch.device(device)
print(f"Using {device} device")

Using cuda device


# Load Model

In [2]:
class DnaCnn(nn.Module):
    def __init__(self, num_kernels=[20, 32, 32], kernel_size=[12,12,12],
                 dropout=0):
        super(DnaCnn, self).__init__()
        self.input_channels=4
        self.num_kernels=num_kernels
        self.kernel_size=kernel_size
        self.dropout=dropout
        self.conv_block = nn.Sequential(
            # first layer
            nn.Conv1d(in_channels=self.input_channels,
                      out_channels=num_kernels[0],
                      kernel_size=kernel_size[0]),
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.MaxPool1d(kernel_size=2),
        )
        # second layer
        self.conv_block.append(nn.Sequential(
            nn.Conv1d(in_channels=self.num_kernels[0],
                      out_channels=num_kernels[1],
                      kernel_size=kernel_size[1]),
            nn.ReLU(),
            #nn.MaxPool1d(kernel_size=2),
            nn.Dropout(p=self.dropout),            
        ))
        # Add a third convolutional layer
        self.conv_block.append(nn.Sequential(
            # second layer
            nn.Conv1d(in_channels=self.num_kernels[1],
                      out_channels=num_kernels[2],
                      kernel_size=kernel_size[2]),
            nn.ReLU(),
            #nn.MaxPool1d(kernel_size=2),
            nn.Dropout(p=self.dropout),            
        ))
        self.regression_block = nn.Sequential(
            nn.Linear(num_kernels[2], num_kernels[2]),
            nn.ReLU(),
            nn.Dropout(p=self.dropout),            
            nn.Linear(num_kernels[2], 1)
            #nn.Sigmoid()
        )            

    def forward(self, x):
        x = self.conv_block(x)
        x,_ = torch.max(x, dim=2)        
        x = self.regression_block(x)
        return x

In [3]:
model = torch.load("model.pth", weights_only=False)
model.to(device)

DnaCnn(
  (conv_block): Sequential(
    (0): Conv1d(4, 20, kernel_size=(12,), stride=(1,))
    (1): ReLU()
    (2): Dropout(p=0, inplace=False)
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Conv1d(20, 32, kernel_size=(12,), stride=(1,))
      (1): ReLU()
      (2): Dropout(p=0, inplace=False)
    )
    (5): Sequential(
      (0): Conv1d(32, 32, kernel_size=(12,), stride=(1,))
      (1): ReLU()
      (2): Dropout(p=0, inplace=False)
    )
  )
  (regression_block): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0, inplace=False)
    (3): Linear(in_features=32, out_features=1, bias=True)
  )
)

# Load Validation Data

In [ ]:
Data = load_data(os.path.join(os.getcwd(), 'Data', 'Parsed_Data'), 
                    test_training_data_to_load=0, 
                    val_data_to_load=math.inf
                    )

training_dataset, testing_dataset, validation_dataset = Data


In [ ]:
print(len(training_dataset))
print(len(testing_dataset))
print(len(validation_dataset))
